<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.cifar10 import load_data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
trans = [       
    "airplane",										
    "automobile",										
    "bird",										
    "cat",										
    "deer",										
    "dog",										
    "frog",										
    "horse",										
    "ship",										
    "truck"
]

In [ ]:
import matplotlib.pyplot as plt
import random
idx = random.randint(0, x_train.shape[0])
print("答案:", trans[y_train[idx][0]])
plt.imshow(x_train[idx])

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
layers = [
    Conv2D(64, 3, padding="same", activation="relu", input_shape=(32, 32, 3)),
    MaxPooling2D(),
    Conv2D(128, 3, padding="same", activation="relu"),
    MaxPooling2D(),
    Conv2D(256, 3, padding="same", activation="relu"),
    GlobalAveragePooling2D(),
    Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
x_train_norm = x_train / 255
x_test_norm = x_test / 255

In [ ]:
# batch:看幾筆在調整一次(20-200)
# epochs:整份訓練資料看幾遍 (10->60000*10)
# verbose: 0(quiet) 1(default) 2(我喜歡的)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint("cnn.h5", save_best_only=True),
    EarlyStopping(patience=10, restore_best_weights=True)
]
model.fit(x_train_norm,
     y_train,
     validation_split=0.1,
     batch_size=200,
     epochs=100,
     verbose=2,
     callbacks=callbacks)

In [ ]:
model.evaluate(x_test_norm, y_test)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
pre = model.predict_classes(x_test_norm)
print(pre.shape)
print(y_test.shape)
y_test_reshape = y_test.reshape(-1)
print(y_test_reshape.shape)
mat = confusion_matrix(y_test_reshape, pre)
pd.DataFrame(mat, 
       columns=["{}(預測)".format(trans[i]) for i in range(10)],
       index=["{}(真實)".format(trans[i]) for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test_reshape)[0]
idx = idx[:200]
img, label, pred = x_test[idx], y_test_reshape[idx], pre[idx]

plt.figure(figsize=(16, 48))
width = 10
height = len(img) // width + 1
for i in range(len(img)):
    plt.subplot(height, width, i+1)
    t = "[T]:{}\n[P]:{}".format(trans[label[i]], trans[pred[i]])
    plt.title(t)
    plt.axis("off")
    plt.imshow(img[i])

In [ ]:
import requests
from PIL import Image

url = input("輸入網址:")
# stream=True: 檔案傳回來, 不加純文字
# stream=True: response.raw stream=False: response.text
# verify=False: 別跟我說不安全
response = requests.get(url, stream=True, verify=False)
img = Image.open(response.raw).resize((32, 32))
test = np.array(img).reshape(1, 32, 32, 3) / 255
p = model.predict(test)[0]
for n, prob in zip(trans, p):
    print(n, "的機率:", round(prob, 3))
ans = model.predict_classes(test)[0]
print("最高機率:", trans[ans])
plt.imshow(img)